In [1]:
! pip3 install -U scikit-learn scipy matplotlib
! pip3 install -U shap 
! pip3 install -U lime 
! pip3 install -U xgboost 
! pip3 install -U eli5 
! pip3 install -U seaborn 

! pip3 install ipywidgets

! pip3 install lightgbm

! pip3 install mlxtend

Requirement already up-to-date: scikit-learn in /home/ava/.local/lib/python3.8/site-packages (0.24.1)
Requirement already up-to-date: scipy in /home/ava/.local/lib/python3.8/site-packages (1.6.0)
Requirement already up-to-date: matplotlib in /home/ava/.local/lib/python3.8/site-packages (3.3.4)
Requirement already up-to-date: shap in /home/ava/.local/lib/python3.8/site-packages (0.38.1)
Requirement already up-to-date: lime in /home/ava/.local/lib/python3.8/site-packages (0.2.0.1)
Requirement already up-to-date: xgboost in /home/ava/.local/lib/python3.8/site-packages (1.3.3)
Requirement already up-to-date: eli5 in /home/ava/.local/lib/python3.8/site-packages (0.11.0)
Requirement already up-to-date: seaborn in /home/ava/.local/lib/python3.8/site-packages (0.11.1)


In [2]:
import os

import numpy as np 
import pandas as pd 

import pickle

import itertools 

from eli5 import show_prediction

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, classification_report

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb 
import lightgbm as lgb

import seaborn as sns 

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt 

from mlxtend.plotting import plot_decision_regions
from mlxtend.classifier import EnsembleVoteClassifier

import sys
sys.path.append('..')

from inxai import *
import shap
import lime

In [3]:
path = '../examples/'

def to_pickle(obj, name):
    with open(path + name + '.pickle', 'wb') as f:
        pickle.dump(obj, f)

def from_pickle(name):
    return pd.read_pickle(path + name + '.pickle')


In [4]:
df = pd.read_csv("../examples/data/compass/propublicaCompassRecividism_data_fairml.csv/propublica_data_for_fairml.csv")
print(df.shape)
display(df.columns)
df.head()

(6172, 12)


Index(['Two_yr_Recidivism', 'Number_of_Priors', 'score_factor',
       'Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'African_American',
       'Asian', 'Hispanic', 'Native_American', 'Other', 'Female',
       'Misdemeanor'],
      dtype='object')

,Two_yr_Recidivism,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor
0,0,0,0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0
2,1,4,0,0,1,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,1
4,1,14,1,0,0,0,0,0,0,0,0,0


In [5]:
TARGET_COL = "Two_yr_Recidivism"

X = df.drop([TARGET_COL],axis=1)
y = df[TARGET_COL]

X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.33, random_state=42)

print("TRAIN:")
print(X_train.head())
print("\nTEST:")
print(y_test.head())

TRAIN:
      Number_of_Priors  score_factor  Age_Above_FourtyFive  \
5025                 3             0                     0   
3098                 2             0                     0   
1187                 3             1                     0   
5238                 3             1                     1   
4270                 1             0                     1   

      Age_Below_TwentyFive  African_American  Asian  Hispanic  \
5025                     1                 0      0         0   
3098                     1                 1      0         0   
1187                     0                 1      0         0   
5238                     0                 0      0         1   
4270                     0                 1      0         0   

      Native_American  Other  Female  Misdemeanor  
5025                0      0       0            1  
3098                0      0       1            0  
1187                0      0       0            0  
5238                0

## Modele 

In [6]:
svc_radial = SVC(kernel='rbf',probability=True) # does not work with eli

svc_radial.fit(X_train, y_train)
svc_radial_preds = svc_radial.predict(X_test)

# to_pickle(svc_radial, 'model_svc_radial')

print(accuracy_score(y_test, svc_radial_preds))
print(classification_report(y_test, svc_radial_preds))

# show_prediction(svc_radial, X_train.iloc[1], feature_names = X.columns.tolist(), 
#                 show_feature_values=True)

0.658321060382916
              precision    recall  f1-score   support

           0       0.68      0.71      0.69      1118
           1       0.63      0.60      0.61       919

    accuracy                           0.66      2037
   macro avg       0.65      0.65      0.65      2037
weighted avg       0.66      0.66      0.66      2037



In [7]:
svc_lin = SVC(kernel='linear',probability=True) 
svc_lin.fit(X_train, y_train)
svc_lin_preds = svc_lin.predict(X_test)

# to_pickle(svc_lin, 'model_svc_lin')

print(accuracy_score(y_test, svc_lin_preds))
print(classification_report(y_test, svc_lin_preds))

show_prediction(svc_lin, X_train.iloc[1], feature_names = X.columns.tolist(), 
                show_feature_values=True)

0.6514482081492391
              precision    recall  f1-score   support

           0       0.68      0.68      0.68      1118
           1       0.61      0.61      0.61       919

    accuracy                           0.65      2037
   macro avg       0.65      0.65      0.65      2037
weighted avg       0.65      0.65      0.65      2037



In [8]:
xgbc = xgb.XGBClassifier() 
xgbc.fit(X_train, y_train)
xgbc_preds = xgbc.predict(X_test)

# to_pickle(xgbc, 'model_xgbc')

print(accuracy_score(y_test, xgbc_preds))
print(classification_report(y_test, xgbc_preds))

show_prediction(xgbc, X_train.iloc[1], feature_names = X.columns.tolist(), 
                show_feature_values=True)

[20:13:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


0.6666666666666666
              precision    recall  f1-score   support

           0       0.69      0.73      0.70      1118
           1       0.64      0.60      0.62       919

    accuracy                           0.67      2037
   macro avg       0.66      0.66      0.66      2037
weighted avg       0.67      0.67      0.67      2037



In [9]:
params = {'metric' : 'auc',
          'boosting_type' : 'gbdt',
          'colsample_bytree' : 0.92,
#           'max_depth' : -1,
#           'n_estimators' : 200,
          'min_child_samples': 4, 
#           'min_child_weight': 0.1,
          'subsample': 0.85,
#           'verbose' : -1,
          'num_threads' : 4 }

lgbm = lgb.train(params,                                    # does not work with eli
                 lgb.Dataset(X_train,label=y_train),
#                  2500,
                 valid_sets=lgb.Dataset(X_test,label=y_test),
                 early_stopping_rounds= 15,
                 verbose_eval= 30
                 )

lgbm_preds = lgbm.predict(X_test)
lgbm_preds = (lgbm_preds > 0.5) * 1

# to_pickle(lgbm, 'model_lgbm')

print(accuracy_score(y_test, lgbm_preds))
print(classification_report(y_test, lgbm_preds))

# show_prediction(lgbm, X_train.iloc[1], feature_names = X.columns.tolist(), #estimator is not supported
#                 show_feature_values=True)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000223 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 49
[LightGBM] [Info] Number of data points in the train set: 4135, number of used features: 11
[LightGBM] [Info] Start training from score 0.457074
Training until validation scores don't improve for 15 rounds
Early stopping, best iteration is:
[7]	valid_0's auc: 0.720958
0.678939617083947
              precision    recall  f1-score   support

           0       0.68      0.80      0.73      1118
           1       0.68      0.54      0.60       919

    accuracy                           0.68      2037
   macro avg       0.68      0.67      0.67      2037
weighted avg       0.68      0.68      0.67      2037



In [10]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_preds = rfc.predict(X_test)

# to_pickle(rfc, 'model_rfc')

print(accuracy_score(y_test, rfc_preds))
print(classification_report(y_test, rfc_preds))

show_prediction(rfc, X_train.iloc[1], feature_names = X.columns.tolist(), 
                show_feature_values=True)

0.6661757486499754
              precision    recall  f1-score   support

           0       0.69      0.72      0.70      1118
           1       0.64      0.61      0.62       919

    accuracy                           0.67      2037
   macro avg       0.66      0.66      0.66      2037
weighted avg       0.67      0.67      0.67      2037



## SHAP's

In [11]:
gm = GlobalFeatureMetric()


In [ ]:
shap_svc_radial = generate_per_instance_importances(models=svc_radial, X=X_test.iloc[:, ], y=y_test.iloc[:, ], framework='kernel_shap') 
to_pickle(shap_svc_radial, 'shap_svc_radial')
len(shap_svc_radial)

shap_svc_lin = generate_per_instance_importances(models=svc_lin, X=X_test.iloc[:, ], y=y_test.iloc[:, ], framework='kernel_shap') 
to_pickle(shap_svc_lin, 'shap_svc_lin')
len(shap_svc_lin)

# shap_xgbc = generate_per_instance_importances(models=xgbc, X=X_test.iloc[:, ], y=y_test.iloc[:, ], framework='kernel_shap') 
# to_pickle(shap_xgbc, 'shap_xgbc')
len(shap_xgbc)

# shap_lgbm = generate_per_instance_importances(models=lgbm, X=X_test.iloc[0:10, ], y=y_test.iloc[0:10, ], framework='kernel_shap') # nie dziala
# to_pickle(shap_lgbm, 'shap_lgbm')
# len(shap_lgbm)

shap_rfc = generate_per_instance_importances(models=rfc, X=X_test.iloc[:, ], y=y_test.iloc[:, ], framework='kernel_shap')  
to_pickle(shap_rfc, 'shap_rfc')
len(shap_rfc)


Using 2037 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/2037 [00:00<?, ?it/s]

In [ ]:
# lime_svc_radial = generate_per_instance_importances(models=svc_radial, X=X_test, y=y_test, framework='lime') # does not work 
# to_pickle(lime_svc_radial, 'lime_svc_radial')
# lime_svc_radial

# lime_svc_lin = generate_per_instance_importances(models=svc_lin, X=X_test, y=y_test, framework='lime') # does not work 
# to_pickle(lime_svc_lin, 'lime_svc_lin')
# lime_svc_lin

# lime_xgbc = generate_per_instance_importances(models=xgbc, X=X_test, y=y_test, framework='lime') # does not work 
# to_pickle(lime_xgbc, 'lime_xgbc')
# lime_xgbc

# lime_lgbm = generate_per_instance_importances(models=lgbm, X=X_test, y=y_test, framework='lime') # does not work 
# to_pickle(lime_lgbm, 'lime_lgbm')
# lime_lgbm

# lime_rfc = generate_per_instance_importances(models=rfc, X=X_test, y=y_test, framework='lime') # does not work 
# to_pickle(lime_rfc, 'lime_lgbm')
# lime_rfc


In [12]:
shap_xgbc = from_pickle('shap_xgbc')
len(shap_xgbc)


2037

In [ ]:
1 + 1 